In [9]:
import requests
import time
from datetime import datetime, timedelta
import pandas as pd
import calendar

In [2]:
# Define the API key
api_key = '6e93a88c176c1843cf9580ab1a700da0'

In [5]:
# Define the cities with latitude & longitude
cities = {
    'Berlin': (52.5200, 13.4050),
    'Madrid': (40.4168, -3.7038),
    'Paris': (48.8566, 2.3522),
    'Vienna': (48.2082, 16.3738),
    'Hamburg': (53.5511, 9.9937),
    'Bucharest': (44.4268, 26.1025),
    'Budapest': (47.4979, 19.0402),
    'Barcelona': (41.3851, 2.1734),
    'Munich': (48.1351, 11.5820),
    'Prague': (50.0755, 14.4378),
    'Milan': (45.4642, 9.1900),
    'Sofia': (42.6977, 23.3219),
    'Cologne': (50.9375, 6.9603),
    'Stockholm': (59.3293, 18.0686),
    'Amsterdam': (52.3676, 4.9041)
}

# Function to get historical air quality data
def get_historical_air_quality_data(api_key, lat, lon, start, end):
    url = f'http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start}&end={end}&appid={api_key}'
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return None

In [6]:
# Convert date to Unix timestamp
def date_to_unix(date):
    return int(time.mktime(date.timetuple()))

In [10]:
def add_months(sourcedate, months):
    month = sourcedate.month - 1 + months
    year = int(sourcedate.year + month / 12 )
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year, month)[1])
    return datetime(year, month, day)

In [24]:
# Collect data for each city from 01 January 2023 to 31 December 2023
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 2, 28)
#delta = timedelta(days=1)

In [25]:
# Create an empty DataFrame
columns = ['City', 'Timestamp', 'Date', 'AQI']
air_quality = pd.DataFrame(columns=columns)

In [ ]:
for city, (lat, lon) in cities.items():
    print(f'Collecting data for {city}')
    current_date = start_date

    while current_date < end_date:
        # Start of the current month
        start_timestamp = date_to_unix(current_date)

        # Start of the next month
        next_month_date = add_months(current_date, 1)
        end_timestamp = date_to_unix(next_month_date)



In [33]:
import requests
import time
from datetime import datetime, timedelta
import pandas as pd
import calendar
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)

# Define the API key
api_key = '6e93a88c176c1843cf9580ab1a700da0'

# Define the city coordinates
cities = {
    'Berlin': (52.5200, 13.4050),
    'Madrid': (40.4168, -3.7038),
    'Paris': (48.8566, 2.3522),
    'Vienna': (48.2082, 16.3738),
    'Hamburg': (53.5511, 9.9937),
    'Bucharest': (44.4268, 26.1025),
    'Budapest': (47.4979, 19.0402),
    'Barcelona': (41.3851, 2.1734),
    'Munich': (48.1351, 11.5820),
    'Prague': (50.0755, 14.4378),
    'Milan': (45.4642, 9.1900),
    'Sofia': (42.6977, 23.3219),
    'Cologne': (50.9375, 6.9603),
    'Stockholm': (59.3293, 18.0686),
    'Amsterdam': (52.3676, 4.9041)
}

# Function to get historical air quality data
def get_historical_air_quality_data(api_key, lat, lon, start, end):
    url = f'http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start}&end={end}&appid={api_key}'
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        logging.error(f"Error fetching data: {e}")
        return None

# Convert date to Unix timestamp
def date_to_unix(date):
    return int(time.mktime(date.timetuple()))

# Helper function to add a month
def add_months(sourcedate, months):
    month = sourcedate.month - 1 + months
    year = int(sourcedate.year + month / 12)
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year, month)[1])
    return datetime(year, month, day)

# Collect data for each city from 01 January 2023 to 31 December 2023
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)

# Create an empty DataFrame
columns = ['City', 'Timestamp', 'Date', 'AQI']
air_quality = pd.DataFrame(columns=columns)

for city, (lat, lon) in cities.items():
    logging.info(f'Collecting data for {city}')
    current_date = start_date

    while current_date < end_date:
        # Start of the current month
        start_timestamp = date_to_unix(current_date)

        # Start of the next month
        next_month_date = add_months(current_date, 1)
        end_timestamp = date_to_unix(next_month_date)

        data = get_historical_air_quality_data(api_key, lat, lon, start_timestamp, end_timestamp)

        if data:
            rows = []
            for entry in data['list']:
                aqi = entry['main']['aqi']
                timestamp = entry['dt']
                date = datetime.fromtimestamp(timestamp)

                # Prepare the row to append
                rows.append({
                    'City': city,
                    'Timestamp': timestamp,
                    'Date': date,
                    'AQI': aqi
                })

            # Append the data to the DataFrame
            air_quality = pd.concat([air_quality, pd.DataFrame(rows)], ignore_index=True)

        current_date = next_month_date
        # Optional: Sleep to respect rate limits
        time.sleep(1)  # Adjust sleep duration as needed

# Print the first few rows of the DataFrame
print(air_quality.head())

# Optional: Save the DataFrame to a CSV file
air_quality.to_csv('air_quality_data_2023_monthly.csv', index=False)


     City   Timestamp                Date AQI
0  Berlin  1672531200 2023-01-01 00:00:00   1
1  Berlin  1672534800 2023-01-01 01:00:00   1
2  Berlin  1672538400 2023-01-01 02:00:00   1
3  Berlin  1672542000 2023-01-01 03:00:00   1
4  Berlin  1672545600 2023-01-01 04:00:00   1


In [35]:
air_quality

,City,Timestamp,Date,AQI
0,Berlin,1672531200,2023-01-01 00:00:00,1
1,Berlin,1672534800,2023-01-01 01:00:00,1
2,Berlin,1672538400,2023-01-01 02:00:00,1
3,Berlin,1672542000,2023-01-01 03:00:00,1
4,Berlin,1672545600,2023-01-01 04:00:00,1
...,...,...,...,...
129952,Amsterdam,1704052800,2023-12-31 20:00:00,2
129953,Amsterdam,1704056400,2023-12-31 21:00:00,2
129954,Amsterdam,1704060000,2023-12-31 22:00:00,2
129955,Amsterdam,1704063600,2023-12-31 23:00:00,2


In [34]:
from google.colab import files
files.download('./air_quality_data_2023.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [39]:
import requests
import time
from datetime import datetime, timedelta
import pandas as pd
import calendar
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)

# Define the API key
api_key = '6e93a88c176c1843cf9580ab1a700da0'

# Define the city coordinates
cities = {
    'Berlin': (52.5200, 13.4050),
    'Madrid': (40.4168, -3.7038),
    'Paris': (48.8566, 2.3522),
    'Vienna': (48.2082, 16.3738),
    'Hamburg': (53.5511, 9.9937),
    'Bucharest': (44.4268, 26.1025),
    'Budapest': (47.4979, 19.0402),
    'Barcelona': (41.3851, 2.1734),
    'Munich': (48.1351, 11.5820),
    'Prague': (50.0755, 14.4378),
    'Milan': (45.4642, 9.1900),
    'Sofia': (42.6977, 23.3219),
    'Cologne': (50.9375, 6.9603),
    'Stockholm': (59.3293, 18.0686),
    'Amsterdam': (52.3676, 4.9041)
}

# Function to get historical air quality data
def get_historical_air_quality_data(api_key, lat, lon, start, end):
    url = f'http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start}&end={end}&appid={api_key}'
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        logging.error(f"Error fetching data: {e}")
        return None

# Convert date to Unix timestamp
def date_to_unix(date):
    return int(time.mktime(date.timetuple()))

# Helper function to add a month
def add_months(sourcedate, months):
    month = sourcedate.month - 1 + months
    year = int(sourcedate.year + month / 12)
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year, month)[1])
    return datetime(year, month, day)

# Collect data for each city from 01 January 2023 to 31 December 2023
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)

# Create an empty list to store monthly aggregated data
monthly_data = []

for city, (lat, lon) in cities.items():
    logging.info(f'Collecting data for {city}')
    current_date = start_date

    while current_date < end_date:
        # Start of the current month
        start_timestamp = date_to_unix(current_date)

        # Start of the next month
        next_month_date = add_months(current_date, 1)
        end_timestamp = date_to_unix(next_month_date)

        data = get_historical_air_quality_data(api_key, lat, lon, start_timestamp, end_timestamp)

        if data:
            # Aggregate AQI for the month
            total_aqi = sum(entry['main']['aqi'] for entry in data['list'])
            num_entries = len(data['list'])
            if num_entries > 0:
                avg_aqi = round(total_aqi / num_entries)
            else:
                avg_aqi = None

            # Append aggregated data to monthly_data list
            monthly_data.append({
                'City': city,
                'Month': current_date.strftime('%Y-%m'),  # Format as YYYY-MM
                'Avg AQI': avg_aqi
            })

        current_date = next_month_date
        # Optional: Sleep to respect rate limits
        time.sleep(1)  # Adjust sleep duration as needed

# Convert monthly_data list to DataFrame
air_quality_monthly = pd.DataFrame(monthly_data)

# Print the first few rows of the DataFrame
print(air_quality_monthly.head())

# Save the DataFrame to a CSV file
air_quality_monthly.to_csv('air_quality_data_2023_monthly_aggregated.csv', index=False)


     City    Month  Avg AQI
0  Berlin  2023-01        1
1  Berlin  2023-02        2
2  Berlin  2023-03        2
3  Berlin  2023-04        2
4  Berlin  2023-05        2


In [40]:
air_quality_monthly.to_csv('./air_quality_data.csv')

In [41]:
from google.colab import files
files.download('./air_quality_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>